- url
- 방법별
- 상황별
- 재료별
- 종류별
- 제목
- 인분
- 시간
- 재료
- 레시피
- 조회수

In [ ]:
import requests,json
from bs4 import BeautifulSoup
import pandas as pd
import re

def clean_json_string(json_string):
    # Remove invalid control characters using regular expression
    cleaned_json_string = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', json_string)
    return cleaned_json_string

# 종류별
# 상황별 <- 만약 데이터프레임에 있는 종류별에 가져왔던 레시피일 경우, 상황별 이름에 데이터프레임 쓰고 넘어가고 없는 것은 새로 쓰기
# 방법별 <- 만약 ""


cat4 = {'밑반찬': '63', '메인반찬': '56', '국/탕': '54', '찌개': '55', '디저트': '60', '면/만두': '53',
           '밥/죽/떡': '52', '양식': '65', '샐러드': '64', '스프': '68'}  # 종류별
cat1 = {'볶음': '6', '끓이기': '1', '부침': '7', '조림': '36', '무침': '41', '비빔': '42',
             '찜': '8', '절임': '10', '튀김': '9', '삶기': '38', '굽기': '67'}  # 방법별
cat3 = {'소고기': '70', '돼지고기': '71', '닭고기': '72', '육류': '23', '채소류': '28', '해물류': '24',
                 '달걀/유제품': '50', '쌀': '47', '밀가루': '32', '건어물류': '25', '버섯류': '31', '과일류': '48',
                 '곡류': '26'}  # 재료별
cat2 = {'일상': '12', '초스피드': '18', '손님접대': '13', '술안주': '19', '다이어트': '21',
                '도시락': '15', '영양식': '43', '간식': '17', '야식': '45', '명절': '44'}  # 상황별



# URL 리스트업 하는 것 -> 각 URL에 정보 크롤링하는 것

# https://www.10000recipe.com/recipe/list.html?cat4=63&order=reco&page=1
# https://www.10000recipe.com/recipe/list.html?cat3=72&order=reco&page=1

category_dic = {'종류별':'cat4','방법별':'cat1','재료별':'cat3','상황별':'cat2'}

# 데이터 프레임으로 만드는 것, subcategory들 category에 맞게 정리되어져야 함


# 만개의 레시피에서 각 분류별로 모든페이지의 레시피 url 리스트업
def food_url_list(category, subcategory): # category: str, subcategory: str

  category_code = category_dic[category]
  subcategory_num = globals()[category_code][subcategory]
  page_num = 1
  page_max_num = 25
  food_url_ids = []

  while page_num <= page_max_num:
    url = f"https://www.10000recipe.com/recipe/list.html?{category_code}={subcategory_num}&order=reco&page={page_num}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        food_list = soup.find_all(attrs={'class':'common_sp_link'})
        # print(food_list)
        food_url_ids_temp = [food['href'].split('/')[-1] for food in food_list]
        food_url_ids.extend(food_url_ids_temp)
        page_num += 1
    else :
        print("HTTP response error :", response.status_code)
        break
  return food_url_ids

# for_loop
# if 기존 리스트에 있는 것이면 타입별에 마킹하고 넘어가도록
# 각 레시피 url에서 food info 가져오며 사전형태로 데이터 구축하기
def food_url_info(food_url_id, category, subcategory):
  url = f'https://www.10000recipe.com/recipe/{food_url_id}'
  response = requests.get(url)
  if response.status_code == 200:
      html = response.text
      soup = BeautifulSoup(html, 'html.parser')
  else :
    print("HTTP response error :", response.status_code)

  food_info = soup.find(attrs={'type':'application/ld+json'})
  # print(food_info)
  try:
    result = json.loads(food_info.text)
  except:
    cleaned_json_string = clean_json_string(food_info.text)
    result = json.loads(cleaned_json_string)
  name = result['name']
  try:
    recipeYield = result['recipeYield']
  except:
    recipeYield = ""
  try:
    totalTime = result['totalTime']
  except:
    totalTime = ""
  ingredient = ','.join(result['recipeIngredient'])
  try:
    recipe_list = [result['recipeInstructions'][i]['text'] for i in range(len(result['recipeInstructions']))]
    recipe = ""
    for i in range(len(recipe_list)):
      recipe += f'{i+1}. ' + recipe_list[i] + '<br>'
  except:
    recipe = "웹페이지 참조"
  foodtype = ""
  foodsituation=""
  foodingredient = ""
  foodmethod = ""
  if category == "종류별":
    foodtype = subcategory
  elif category == "상황별":
    foodsituation = subcategory
  elif category == "재료별":
    foodingredient = subcategory
  elif category == "방법별":
    foodmethod = subcategory
  else:
    print("Error input")

  res = {
        'url': url,
        '방법별': foodmethod,
        '상황별': foodsituation,
        '재료별': foodingredient,
        '종류별': foodtype,
        '요리제목': name,
        '인분': recipeYield,
        '시간': totalTime,
        '재료': ingredient,
        '레시피': recipe,
    }

  return res

In [ ]:
# 종류별에 subcategory 하나씩 가져오기
df = pd.DataFrame()
save_df = '/content/drive/MyDrive/KITA_1026/project/chatbot project/data/recipe_main.csv'

category = "종류별"
subcategory = "메인반찬"

crawl_list = food_url_list(category,subcategory)
for food_url_id in crawl_list:
  print(food_url_id)
  try:
    food_dict = food_url_info(food_url_id, category, subcategory)
    df_dict = pd.DataFrame(food_dict, index=[0])
    df = pd.concat([df, df_dict], ignore_index=True)
  except:
    continue

df.to_csv(save_df, encoding='utf-8-sig', index=False)

df

7003451
6966445
7020729
6915088
6994439
7020726
7020725
7002427
7020721
6975337
6897374
6980995
6944255
6913952
6881099
6881815
6868389
6939980
7017097
6922700
6983253
6894323
6899265
6924130
6918383
6954484
7017130
7004934
6922190
6895591
6898497
6976511
6918977
6912739
6990828
6872474
6884169
6847506
6874866
6854799
6842456
6994661
7019124
6868435
6887603
6887226
6918300
6916655
6867358
7013169
6924357
6843348
6811239
6879543
6882435
6927971
6980900
7004170
6977567
6925795
6844208
6861562
6863792
6880694
6867023
6882848
6909721
6965550
7018288
6906436
6933144
6861235
7001408
6855088
6916403
6942026
6952146
7015067
6984865
6894649
6955878
6873941
6875013
7019040
7019729
6874259
6887181
6897328
6980604
6847798
6865791
6894964
6962360
6963453
6835918
6990879
6878619
6883479
6922742
7017225
6882107
6882843
6934182
6999755
6885922
6962887
6835653
6899911
6921400
6932754
6934142
6896720
6894890
7015045
6940325
6860267
6957721
6880674
6922879
6959353
6961293
6834667
6929819
6843483
6908836


,url,방법별,상황별,재료별,종류별,요리제목,인분,시간,재료,레시피
0,https://www.10000recipe.com/recipe/7003451,,,,메인반찬,"간단하게 만드는 밥도둑, 팽이버섯두부조림",2 servings,PT20M,"두부 1모,팽이버섯 1봉,대파 15cm,간장 3숟가락,맛술 1숟가락,다진 마늘 1숟...",1. 두부는 약 1.5cm두깨로 썰어요.<br>2. 팽이버섯은 먹기 좋게 가닥가닥 ...
1,https://www.10000recipe.com/recipe/6966445,,,,메인반찬,파프리카달걀볶음:초간단 저탄고지 다이어트 단백질식사,1 servings,PT10M,"파프리카 1개,달걀 3개,양파,현미유 조금,소금 조금,후추 조금,파슬리가루",1. 파프리카와 양파는 한입 크기로 썬다.<br>2. 달걀은 풀어준다. ​<br>...
2,https://www.10000recipe.com/recipe/7020729,,,,메인반찬,브로컬리줄기 활용 ~ 냉파 야채 잡채 만드는법,4 servings,PT30M,"브로콜리 줄기 2개,당면 300g,양파 1/2개,당근 1/2개,미나리 70g,다진 ...",1. 당면을 물에 30분 이상 담가 주세요. 물에 불려 부드러워지면 삶는 시간을 단...
3,https://www.10000recipe.com/recipe/6915088,,,,메인반찬,밥도둑 반찬 고추장 달걀조림 만들기!TV 알토란 레시피,3 servings,PT20M,"달걀 10개,양파 1개,청양고추 3개,쪽파 또는 대파 약간,멸치육수 1.5컵,고추장...","1. 먼저 물에 소금1T,식초1T를 섞어 달걀을 7-8분 반숙으로 삶아 줍니다.<b..."
4,https://www.10000recipe.com/recipe/6994439,,,,메인반찬,두부조림보다 부드러운 매콤 순두부 요리,2 servings,PT20M,"순두부 1봉,표고버섯 2개,양파 1/2개,대파 1/2대,청양고추 1개,참기름 1숟가...",1. 순두부는 1~2cm 두께로 썬다.<br>2. 표고버섯은 약 0.5cm 두께로 ...
...,...,...,...,...,...,...,...,...,...,...
987,https://www.10000recipe.com/recipe/6837664,,,,메인반찬,집밥 백선생 두부김치 만들기!!!!!,2 servings,PT30M,"두부 1모,신김치 1/4쪽,삼겹살 250g,양파 1개,당근 약간,대파 1대,고추 2...",1. 저는 고기는 삼겹살을 준비했어요! 먼저 삼겹살을 이렇게 팬바닥에 깔아줍니다 목...
988,https://www.10000recipe.com/recipe/6872095,,,,메인반찬,다이어트 반찬 두부 스테이크 만들기,4 servings,PT15M,"두부 1모,파프리카 1/2개,양파 1/2개,달걀 1개,밀가루 1컵,소금 조금,후추 조금","1. 두부는 물기를 최대한 제거해서 준비합니다 양파, 파프리카는 썰어서 준비해주고..."
989,https://www.10000recipe.com/recipe/6541357,,,,메인반찬,양파 크래미전 레시피,,,"양파,크래미,계란",1. 양파는 잘게 다져주었어요<br>2. 크래미는 위에 보이는 8개 전부 사용했어요...
990,https://www.10000recipe.com/recipe/6832783,,,,메인반찬,백종원의 '콩나물 불고기',2 servings,PT30M,"대패삼겹살 300g,콩나물 200g,당근 1/3개,깻잎 10장,대파 1대,청양고추 ...","1. 양파는 채 썰고 당근, 깻잎, 대파는 먹기 좋은 크기로 썬다<br>2. 콩나물..."
